In [1]:
from pyspark.sql import SparkSession
import os

In [2]:
data_dir = '../data/Spark_DataFrames/'

spark = SparkSession.builder.appName('ops').getOrCreate()

In [3]:
df = spark.read.csv(os.path.join(data_dir, 'appl_stock.csv'), inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [10]:
df.head(10)[0]

Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

## Filter and Select operations

In [16]:
df.filter('Close < 500').select(['Open', 'Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [21]:
# df.filter(df['Close'] < 500).select('Volume').show()
df.filter((df['Close'] < 200) & (df['Open'] > 200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [23]:
## We are gonna usually use collect() method instead of show() in real world. This will return a list.

result = df.filter(df['Low'] == 197.16).collect()

row = result[0].asDict()['Volume']
print(row)

220441900


## Groupby operation


In [24]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [25]:
df = spark.read.csv(os.path.join(data_dir, 'sales_info.csv'), inferSchema=True, header=True )

In [27]:
df.show()
df.printSchema()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [35]:
# df.groupBy('Company').mean().show()
# df.groupBy('Company').sum().show()
# df.groupBy('Company').min().show()
df.groupBy('Company').max().show()
# df.groupBy('Company').count().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [33]:
df.agg({'Sales': 'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [34]:
group_data = df.groupBy('Company')
group_data.agg({'Sales': 'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [39]:
## Some functions from spark

from pyspark.sql.functions import countDistinct, avg, stddev

df.select(countDistinct('Sales')).show()

df.select(avg('Sales').alias('Average Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [41]:
## Sort the dataframe

df.orderBy('Sales').show()
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## Dealing with Missing Data

In [43]:
spark = SparkSession.builder.appName('miss').getOrCreate()

df = spark.read.csv(os.path.join(data_dir, 'ContainsNull.csv'), inferSchema=True, header=True)

In [44]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [47]:
# df.na.drop().show()
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [54]:
# df.na.drop(how='any').show()
# df.na.drop(how='all').show()
df.na.drop(subset = ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [56]:
## Filling missing values

# Fill a column with a fixed value
df.na.fill('NO NAME', subset=['Name']).show()

# Fill a column with mean value
from pyspark.sql.functions import mean

mean_val = df.select(mean(df['Sales'])).collect()[0][0]
df.na.fill(mean_val, subset=['Sales']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|NO NAME| null|
|emp3|NO NAME|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Dates and Timestamps

In [57]:
spark = SparkSession.builder.appName('dates').getOrCreate()

In [58]:
df = spark.read.csv(os.path.join(data_dir, 'appl_stock.csv'), inferSchema=True, header=True)

In [59]:
df.head(1)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [61]:
df.select(['Date', 'Open']).show() 

+----------+------------------+
|      Date|              Open|
+----------+------------------+
|2010-01-04|        213.429998|
|2010-01-05|        214.599998|
|2010-01-06|        214.379993|
|2010-01-07|            211.75|
|2010-01-08|        210.299994|
|2010-01-11|212.79999700000002|
|2010-01-12|209.18999499999998|
|2010-01-13|        207.870005|
|2010-01-14|210.11000299999998|
|2010-01-15|210.92999500000002|
|2010-01-19|        208.330002|
|2010-01-20|        214.910006|
|2010-01-21|        212.079994|
|2010-01-22|206.78000600000001|
|2010-01-25|202.51000200000001|
|2010-01-26|205.95000100000001|
|2010-01-27|        206.849995|
|2010-01-28|        204.930004|
|2010-01-29|        201.079996|
|2010-02-01|192.36999699999998|
+----------+------------------+
only showing top 20 rows



In [63]:
from pyspark.sql.functions import (dayofmonth, hour, dayofyear,
                                   month, year, weekofyear,
                                   format_number, date_format)

df.select(dayofmonth(df['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [71]:
## Show average closing price per year

# df.select(year(df['Date'])).show()
new_df = df.withColumn('year', year(df['Date']))
result = new_df.groupBy('Year').mean().select(['Year', 'avg(Close)'])

In [76]:
new = result.withColumnRenamed('avg(Close)', 'Average Closing Price')
# new = new.select(['Year', format_number('Average Closing Price', 2).alias('Average Closing Price')])
new = new.select(['Year', format_number('Average Closing Price', 2).alias('Average Closing Price')])
new.show()

+----+---------------------+
|Year|Average Closing Price|
+----+---------------------+
|2015|               120.04|
|2013|               472.63|
|2014|               295.40|
|2012|               576.05|
|2016|               104.60|
|2010|               259.84|
|2011|               364.00|
+----+---------------------+

